In [ ]:
import pandas as pd
import numpy as np
import pandas as pd

from OptimalHedging.GBM import GBMSimulator  # conforme você pediu

# -----------------------------
# 0) Parâmetros
# -----------------------------
M     = 1000
N     = 252
K     = 12.0
S0    = 10.0
T     = 1.0
t0    = 0.0
mu    = 0.05
sigma = 0.2

gamma = 2.0
a     = 2.0
k     = 2.0
beta  = 0.95
U = 10.0

np.random.seed(123)

t_idx = 0.2

# -----------------------------
# 1) Instancia + simula S e H
# -----------------------------
sim = GBMSimulator(
    M=M, N=N, K=K, S0=S0, T=T, t0=t0, mu=mu, sigma=sigma
)

sim.simulate_S()
sim.simulate_H()

# -----------------------------
# 2) Riscos: definição de kwargs
# -----------------------------
risk_specs = {
    "ele" : {"a": a},
    "elw" : {"k": k, "U": U},
    "entl": {"gamma": gamma, "U": U},
    "ente": {"gamma": gamma, "a": a, "U": U},
    "entw": {"gamma": gamma, "k": k, "U": U},
    "esl" : {"beta": beta},
}

# Ordem de avaliação nas colunas (igual ao print do exemplo)
eval_order = ["ele", "elw", "entl", "ente", "entw", "esl"]

# -----------------------------
# 3) Loop: otimiza e avalia
# -----------------------------
rows = []
hedges = {}

for risk_type, risk_kwargs in risk_specs.items():
    MR, info = sim.compute_MR(
        t_idx=t_idx,
        risk_type=risk_type,
        risk_kwargs=risk_kwargs,
        max_iter=20,
        tol=1e-4,
        alpha=1e-3,
        verbose=True
    )

    rows.append({
        "Model": "GBM",
        "Risk": risk_type,
        "t_idx": t_idx,
        "MR": MR,
        "rho_T": info["rho_T"],
        "rho_t": info["rho_t"],
    })

df_MR = pd.DataFrame(rows)

pd.set_option("display.float_format", lambda x: f"{x:.6f}")
display(df_MR)


In [ ]:
# fixa K e hedge (para não misturar efeitos)
sim.K = 10.0
sim.simulate_H()
h, _ = sim.optimize_hedge(
    risk_type="esl",
    risk_kwargs={"beta": 0.95},
    t_idx=0.0,
    verbose=False
)

# dois tempos distintos
t1 = 0.2
t2 = 0.8

L_t1 = sim.forward_PL(h, L0=0.0, t_start=t1)
L_t2 = sim.forward_PL(h, L0=0.0, t_start=t2)

# plota um cenário
import matplotlib.pyplot as plt
plt.plot(L_t1[0], label=f"t={t1}")
plt.plot(L_t2[0], label=f"t={t2}")
plt.legend()
plt.title("P&L paths (same hedge, different t)")
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

from OptimalHedging.GBM import GBMSimulator

# ============================================================
# 0) Parâmetros do experimento
# ============================================================
# Processo
M     = 1000
N     = 252
S0    = 10.0
T     = 1.0
t0    = 0.0
mu    = 0.05
sigma = 0.2

# Risco (ESL)
risk_type   = "esl"
beta        = 0.95
risk_kwargs = {"beta": beta}

# Malhas (tempo e moneyness)
# Você pediu t em [0, 1]. Observação: t=T pode ficar degenerado (MR ~ 0/ruído numérico).
n_t = 21
n_k = 21
t_grid = np.linspace(t0, T, n_t)                # inclui T
K_grid = np.linspace(S0 * 0.8, S0 * 1.6, n_k)   # garante moneyness em [0.8, 1.6]
# equivalente a K em [6.25, 12.5] quando S0=10

# Otimização
max_iter = 20
tol      = 1e-4
alpha    = 1e-3
verbose  = False

np.random.seed(123)

# ============================================================
# 1) Simulação do GBM (fixa para toda a superfície)
# ============================================================
# K inicial é placeholder; vamos sobrescrever dentro do loop
sim = GBMSimulator(M=M, N=N, K=K_grid[0], S0=S0, T=T, t0=t0, mu=mu, sigma=sigma)
sim.simulate_S()

# ============================================================
# 2) Loop duplo: (K, t) -> MR
# ============================================================
rows = []
for K in K_grid:
    sim.K = float(K)
    sim.simulate_H()  # payoff depende de K

    for t in t_grid:
        print(f"K={K:.4f} | m={S0/K:.3f} | t={t:.3f}")
        MR, info = sim.compute_MR(
            t_idx=float(t),              # aqui é TEMPO (não índice)
            risk_type=risk_type,
            risk_kwargs=risk_kwargs,
            max_iter=max_iter,
            tol=tol,
            alpha=alpha,
            verbose=verbose
        )

        rows.append({
            "t": float(t),
            "K": float(K),
            "moneyness": float(S0 / K),
            "MR": float(MR),
            "rho_T": float(info["rho_T"]),
            "rho_t": float(info["rho_t"]),
        })

df = pd.DataFrame(rows)

# ============================================================
# 3) Pivot para malhas 2D (moneyness x tempo)
# ============================================================
# Linhas = moneyness, colunas = t
pivot = df.pivot_table(index="moneyness", columns="t", values="MR", aggfunc="mean")

m_vals = pivot.index.to_numpy()
t_vals = pivot.columns.to_numpy()

T_mesh, M_mesh = np.meshgrid(t_vals, m_vals)
MR_mesh = pivot.to_numpy()

# ============================================================
# 4) Gráfico 3D (superfície)
# ============================================================
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection="3d")

ax.plot_surface(T_mesh, M_mesh, MR_mesh, linewidth=0, antialiased=True)

ax.set_xlabel("t")
ax.set_ylabel("S0 / K (moneyness)")
ax.set_zlabel("MR (ESL)")

ax.set_title(f"Maturity Risk Surface — ESL (beta={beta})")

plt.tight_layout()
plt.show()

# ============================================================
# 5) (Opcional) salvar resultados
# ============================================================
# df.to_csv("mr_surface_esl.csv", index=False)
# pivot.to_csv("mr_surface_esl_pivot.csv")


In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

# força renderização no navegador (independente de Jupyter)
pio.renderers.default = "browser"

fig = go.Figure(
    data=[
        go.Surface(
            x=T_mesh,
            y=M_mesh,
            z=MR_mesh,
            colorscale="RdBu",
            reversescale=True,
            opacity=0.95,
            contours=dict(
                z=dict(
                    show=True,
                    usecolormap=True,
                    highlightcolor="black",
                    project=dict(z=True)
                )
            ),
            showscale=True
        )
    ]
)

fig.update_layout(
    title="Maturity Risk Surface (ESL)",
    width=950,
    height=750,
    scene=dict(
        xaxis_title="t",
        yaxis_title="S0 / K (moneyness)",
        zaxis_title="MR",
        xaxis=dict(backgroundcolor="white"),
        yaxis=dict(backgroundcolor="white"),
        zaxis=dict(backgroundcolor="white"),
        camera=dict(
            eye=dict(x=1.6, y=1.6, z=0.9)
        )
    )
)

fig.show()


In [2]:
import pandas as pd
import numpy as np

from OptimalHedging.JumpDiff import JumpDiffusionSimulator

# -----------------------------
# 0) Parameters
# -----------------------------
M     = 1000
N     = 252
K     = 15.0
S0    = 10.0
T     = 1.0
t0    = 0.0
mu    = 0.05
sigma = 0.2

lam   = 0.5
meanJ = -0.03
stdJ  = 0.12

gamma = 1.0
a     = 1.0
k     = 2.0
beta  = 0.95
U     = 10.0

np.random.seed(123)

t_idx = 0.8

# -----------------------------
# 1) Instantiate + simulate S and H
# -----------------------------
sim = JumpDiffusionSimulator(
    M=M, N=N, K=K, S0=S0, T=T, t0=t0, mu=mu, sigma=sigma,
    lam=lam, meanJ=meanJ, stdJ=stdJ
)

sim.simulate_S()
sim.simulate_H(p_t=1, p_x=2, lam_ridge=1e-2)

# -----------------------------
# 2) Risks: kwargs
# -----------------------------
risk_specs = {
    "ele" : {"a": a},
    "elw" : {"k": k, "U": U},
    "entl": {"gamma": gamma, "U": U},
    "ente": {"gamma": gamma, "a": a, "U": U},
    "entw": {"gamma": gamma, "k": k, "U": U},
    "esl" : {"beta": beta},
}

eval_order = ["ele", "elw", "entl", "ente", "entw", "esl"]

# -----------------------------
# 3) Loop: compute MR
# -----------------------------
rows = []

for risk_type, risk_kwargs in risk_specs.items():
    MR, info = sim.compute_MR(
        t_idx=t_idx,
        risk_type=risk_type,
        risk_kwargs=risk_kwargs,
        kind="MinVar",
        max_iter=20,
        tol=1e-4,
        alpha=1e-1,
        verbose=False
    )

    rows.append({
        "Model": "JumpDiffusion",
        "Risk": risk_type,
        "t_idx": t_idx,
        "MR": MR,
        "rho_T": info["rho_T"],
        "rho_t": info["rho_t"],
    })

df_MR = pd.DataFrame(rows)

df_MR["Risk"] = pd.Categorical(df_MR["Risk"], categories=eval_order, ordered=True)
df_MR = df_MR.sort_values("Risk").reset_index(drop=True)

pd.set_option("display.float_format", lambda x: f"{x:.6f}")
display(df_MR)


,Model,Risk,t_idx,MR,rho_T,rho_t
0,JumpDiffusion,ele,0.800000,0.000054,1.000000,1.000054
1,JumpDiffusion,elw,0.800000,0.001943,1.000000,1.001943
2,JumpDiffusion,entl,0.800000,0.002054,0.000000,0.002054
3,JumpDiffusion,ente,0.800000,0.013672,1.000000,1.013672
4,JumpDiffusion,entw,0.800000,0.001927,1.000000,1.001927
5,JumpDiffusion,esl,0.800000,0.125903,0.000000,0.125903


In [ ]:
import numpy as np
import pandas as pd

from OptimalHedging.GBM import GBMSimulator
from OptimalHedging.Heston import HestonSimulator
from OptimalHedging.JumpDiff import JumpDiffusionSimulator

np.random.seed(123)

M     = 1000
N     = 252
S0    = 10.0
K0    = 15.0
T     = 1.0
t0    = 0.0

mu    = 0.05
sigma = 0.2

kappa   = 1.0
theta   = 0.1
sigma_v = 0.1
corr    = 0.7

lam   = 0.5
meanJ = -0.03
stdJ  = 0.12


gamma = 1.0
a     = 1.0
k     = 2.0
beta  = 0.95
U     = 10.0

risk_specs = {
    "ele" : {"a": a},
    "elw" : {"k": k, "U": U},
    "entl": {"gamma": gamma, "U": U},
    "ente": {"gamma": gamma, "a": a, "U": U},
    "entw": {"gamma": gamma, "k": k, "U": U},
    "esl" : {"beta": beta},
}


In [ ]:
# ============================================================
# BLOCO 2 - SURFACE: MR(t, moneyness) for GBM, Heston, Jump
# Saves CSV after EACH (model, risk_type)
# ============================================================
n_t = 21
n_k = 21
t_grid = np.linspace(t0, T, n_t)
K_grid = np.linspace(8.0, 16.0, n_k)

max_iter = 20
tol = 1e-4
alpha = 1e-3
verbose = False
kind = "MinVar"

# -----------------------------
# GBM
# -----------------------------
sim = GBMSimulator(M=M, N=N, S0=S0, K=K0, T=T, t0=t0, mu=mu, sigma=sigma)

for risk_type, risk_kwargs in risk_specs.items():
    rows = []
    total = len(K_grid) * len(t_grid)
    done = 0
    print(f"[SURFACE] model=GBM | risk={risk_type} | total={total}")

    for K in K_grid:
        sim.K = float(K)
        m = float(S0 / K)
        for t in t_grid:
            MR, _ = sim.compute_MR(
                t_idx=float(t),
                risk_type=risk_type,
                risk_kwargs=risk_kwargs,
                kind=kind,
                max_iter=max_iter,
                tol=tol,
                alpha=alpha,
                verbose=verbose
            )
            rows.append({"model": "GBM", "risk_type": risk_type, "t": float(t), "moneyness": m, "MR": float(MR)})
            done += 1
            if done == 1 or done % max(1, total // 20) == 0 or done == total:
                print(f"[SURFACE] model=GBM | risk={risk_type} | {100*done/total:6.2f}%")

    df = pd.DataFrame(rows)
    df.to_csv(f"surface_GBM_{risk_type}.csv", index=False)
    print(f"[SURFACE] saved surface_GBM_{risk_type}.csv | rows={len(df)}")

# -----------------------------
# Heston
# -----------------------------
sim = HestonSimulator(
    M=M, N=N, S0=S0, K=K0, T=T, t0=t0,
    mu=mu, sigma=sigma,
    kappa=kappa, theta=theta, sigma_v=sigma_v, corr=corr
)

for risk_type, risk_kwargs in risk_specs.items():
    rows = []
    total = len(K_grid) * len(t_grid)
    done = 0
    print(f"[SURFACE] model=Heston | risk={risk_type} | total={total}")

    for K in K_grid:
        sim.K = float(K)
        m = float(S0 / K)
        for t in t_grid:
            MR, _ = sim.compute_MR(
                t_idx=float(t),
                risk_type=risk_type,
                risk_kwargs=risk_kwargs,
                kind=kind,
                max_iter=max_iter,
                tol=tol,
                alpha=alpha,
                verbose=verbose
            )
            rows.append({"model": "Heston", "risk_type": risk_type, "t": float(t), "moneyness": m, "MR": float(MR)})
            done += 1
            if done == 1 or done % max(1, total // 20) == 0 or done == total:
                print(f"[SURFACE] model=Heston | risk={risk_type} | {100*done/total:6.2f}%")

    df = pd.DataFrame(rows)
    df.to_csv(f"surface_Heston_{risk_type}.csv", index=False)
    print(f"[SURFACE] saved surface_Heston_{risk_type}.csv | rows={len(df)}")

# -----------------------------
# Jump
# -----------------------------
sim = JumpDiffusionSimulator(
    M=M, N=N, S0=S0, K=K0, T=T, t0=t0,
    mu=mu, sigma=sigma,
    lam=lam, meanJ=meanJ, stdJ=stdJ
)

for risk_type, risk_kwargs in risk_specs.items():
    rows = []
    total = len(K_grid) * len(t_grid)
    done = 0
    print(f"[SURFACE] model=Jump | risk={risk_type} | total={total}")

    for K in K_grid:
        sim.K = float(K)
        m = float(S0 / K)
        for t in t_grid:
            MR, _ = sim.compute_MR(
                t_idx=float(t),
                risk_type=risk_type,
                risk_kwargs=risk_kwargs,
                kind=kind,
                max_iter=max_iter,
                tol=tol,
                alpha=alpha,
                verbose=verbose
            )
            rows.append({"model": "Jump", "risk_type": risk_type, "t": float(t), "moneyness": m, "MR": float(MR)})
            done += 1
            if done == 1 or done % max(1, total // 20) == 0 or done == total:
                print(f"[SURFACE] model=Jump | risk={risk_type} | {100*done/total:6.2f}%")

    df = pd.DataFrame(rows)
    df.to_csv(f"surface_Jump_{risk_type}.csv", index=False)
    print(f"[SURFACE] saved surface_Jump_{risk_type}.csv | rows={len(df)}")


In [ ]:
# ============================================================
# BLOCO 3 - PROCESS SENS (ATM): DeltaMR on (param1,param2), t_idx in {0.2,0.8}
# Saves CSV after EACH (model, risk_type, t_idx)
# ============================================================
t_list = [0.2, 0.8]
max_iter = 20
tol = 1e-4
alpha = 1e-3
verbose = False
kind = "MinVar"

# grids
mu_grid    = np.linspace(0.025, 0.075, 9)
sig_grid   = np.linspace(0.10, 0.40, 9)

kappa_grid = np.linspace(1.0, 4.0, 9)
theta_grid = np.linspace(0.1, 0.3, 9)

meanJ_grid = np.linspace(-0.3, 0.3, 9)
lam_grid   = np.linspace(0.0, 1.0, 9)

# -----------------------------
# GBM: mu x sigma
# -----------------------------
sim = GBMSimulator(M=M, N=N, S0=S0, K=S0, T=T, t0=t0, mu=mu, sigma=sigma)

for risk_type, risk_kwargs in risk_specs.items():
    for t_idx in t_list:
        # baseline (standard mu,sigma)
        sim.mu = float(mu)
        sim.sigma = float(sigma)
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx),
            risk_type=risk_type,
            risk_kwargs=risk_kwargs,
            kind=kind,
            max_iter=max_iter,
            tol=tol,
            alpha=alpha,
            verbose=verbose
        )

        rows = []
        total = len(mu_grid) * len(sig_grid)
        done = 0
        print(f"[SENS] model=GBM | risk={risk_type} | t_idx={t_idx} | total={total}")

        for mu_i in mu_grid:
            for sig_i in sig_grid:
                sim.mu = float(mu_i)
                sim.sigma = float(sig_i)
                MR, _ = sim.compute_MR(
                    t_idx=float(t_idx),
                    risk_type=risk_type,
                    risk_kwargs=risk_kwargs,
                    kind=kind,
                    max_iter=max_iter,
                    tol=tol,
                    alpha=alpha,
                    verbose=verbose
                )
                rows.append({
                    "model": "GBM", "risk_type": risk_type, "t_idx": float(t_idx),
                    "mu": float(mu_i), "sigma": float(sig_i),
                    "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
                })
                done += 1
                if done == 1 or done % max(1, total // 20) == 0 or done == total:
                    print(f"[SENS] model=GBM | risk={risk_type} | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_process_GBM_{risk_type}_t{t_idx}.csv", index=False)
        print(f"[SENS] saved sens_process_GBM_{risk_type}_t{t_idx}.csv | rows={len(df)}")

# -----------------------------
# Heston: kappa x theta
# -----------------------------
sim = HestonSimulator(
    M=M, N=N, S0=S0, K=S0, T=T, t0=t0,
    mu=mu, sigma=sigma,
    kappa=kappa, theta=theta, sigma_v=sigma_v, corr=corr
)

for risk_type, risk_kwargs in risk_specs.items():
    for t_idx in t_list:
        sim.kappa = float(kappa)
        sim.theta = float(theta)
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx),
            risk_type=risk_type,
            risk_kwargs=risk_kwargs,
            kind=kind,
            max_iter=max_iter,
            tol=tol,
            alpha=alpha,
            verbose=verbose
        )

        rows = []
        total = len(kappa_grid) * len(theta_grid)
        done = 0
        print(f"[SENS] model=Heston | risk={risk_type} | t_idx={t_idx} | total={total}")

        for k_i in kappa_grid:
            for th_i in theta_grid:
                sim.kappa = float(k_i)
                sim.theta = float(th_i)
                MR, _ = sim.compute_MR(
                    t_idx=float(t_idx),
                    risk_type=risk_type,
                    risk_kwargs=risk_kwargs,
                    kind=kind,
                    max_iter=max_iter,
                    tol=tol,
                    alpha=alpha,
                    verbose=verbose
                )
                rows.append({
                    "model": "Heston", "risk_type": risk_type, "t_idx": float(t_idx),
                    "kappa": float(k_i), "theta": float(th_i),
                    "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
                })
                done += 1
                if done == 1 or done % max(1, total // 20) == 0 or done == total:
                    print(f"[SENS] model=Heston | risk={risk_type} | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_process_Heston_{risk_type}_t{t_idx}.csv", index=False)
        print(f"[SENS] saved sens_process_Heston_{risk_type}_t{t_idx}.csv | rows={len(df)}")

# -----------------------------
# Jump: meanJ x lam
# -----------------------------
sim = JumpDiffusionSimulator(
    M=M, N=N, S0=S0, K=S0, T=T, t0=t0,
    mu=mu, sigma=sigma,
    lam=lam, meanJ=meanJ, stdJ=stdJ
)

for risk_type, risk_kwargs in risk_specs.items():
    for t_idx in t_list:
        sim.meanJ = float(meanJ)
        sim.lam = float(lam)
        MR0, _ = sim.compute_MR(
            t_idx=float(t_idx),
            risk_type=risk_type,
            risk_kwargs=risk_kwargs,
            kind=kind,
            max_iter=max_iter,
            tol=tol,
            alpha=alpha,
            verbose=verbose
        )

        rows = []
        total = len(meanJ_grid) * len(lam_grid)
        done = 0
        print(f"[SENS] model=Jump | risk={risk_type} | t_idx={t_idx} | total={total}")

        for mJ in meanJ_grid:
            for l in lam_grid:
                sim.meanJ = float(mJ)
                sim.lam = float(l)
                MR, _ = sim.compute_MR(
                    t_idx=float(t_idx),
                    risk_type=risk_type,
                    risk_kwargs=risk_kwargs,
                    kind=kind,
                    max_iter=max_iter,
                    tol=tol,
                    alpha=alpha,
                    verbose=verbose
                )
                rows.append({
                    "model": "Jump", "risk_type": risk_type, "t_idx": float(t_idx),
                    "meanJ": float(mJ), "lam": float(l),
                    "MR": float(MR), "MR0": float(MR0), "DeltaMR": float(MR - MR0)
                })
                done += 1
                if done == 1 or done % max(1, total // 20) == 0 or done == total:
                    print(f"[SENS] model=Jump | risk={risk_type} | t_idx={t_idx} | {100*done/total:6.2f}%")

        df = pd.DataFrame(rows)
        df.to_csv(f"sens_process_Jump_{risk_type}_t{t_idx}.csv", index=False)
        print(f"[SENS] saved sens_process_Jump_{risk_type}_t{t_idx}.csv | rows={len(df)}")


In [ ]:
# ============================================================
# BLOCO 4 - AVERSION SENS (ATM): DeltaMR vs param, t_idx in {0.2,0.8}
# Saves CSV after EACH (model, risk_type, param, t_idx)
# ============================================================
t_list = [0.2, 0.8]
max_iter = 20
tol = 1e-4
alpha = 1e-3
verbose = False
kind = "MinVar"

gamma_grid = np.linspace(0.1, 2.0, 15)
a_grid     = np.linspace(0.5, 2.0, 15)
k_grid     = np.linspace(1.1, 3.0, 15)
beta_list  = [0.9, 0.95, 0.975, 0.99]

# -----------------------------
# GBM
# -----------------------------
sim = GBMSimulator(M=M, N=N, S0=S0, K=S0, T=T, t0=t0, mu=mu, sigma=sigma)

# gamma: entl, ente, entw
for risk_type in ["entl", "ente", "entw"]:
    base_kwargs = dict(risk_specs[risk_type])
    for t_idx in t_list:
        MR0, _ = sim.compute_MR(t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
                                kind=kind, max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose)
        rows = []
        total = len(gamma_grid)
        done = 0
        print(f"[AVERS] model=GBM | risk={risk_type} | param=gamma | t_idx={t_idx} | total={total}")
        for g in gamma_grid:
            rk = dict(base_kwargs); rk["gamma"] = float(g)
            MR, _ = sim.compute_MR(t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
                                   kind=kind, max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose)
            rows.append({"model":"GBM","risk_type":risk_type,"t_idx":float(t_idx),
                         "param_name":"gamma","param_value":float(g),
                         "MR":float(MR),"MR0":float(MR0),"DeltaMR":float(MR-MR0)})
            done += 1
            if done == 1 or done % max(1, total // 10) == 0 or done == total:
                print(f"[AVERS] model=GBM | risk={risk_type} | gamma | t_idx={t_idx} | {100*done/total:6.2f}%")
        df = pd.DataFrame(rows)
        df.to_csv(f"sens_aversion_GBM_{risk_type}_gamma_t{t_idx}.csv", index=False)

# a: ele, ente
for risk_type in ["ele", "ente"]:
    base_kwargs = dict(risk_specs[risk_type])
    for t_idx in t_list:
        MR0, _ = sim.compute_MR(t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
                                kind=kind, max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose)
        rows = []
        total = len(a_grid)
        done = 0
        print(f"[AVERS] model=GBM | risk={risk_type} | param=a | t_idx={t_idx} | total={total}")
        for av in a_grid:
            rk = dict(base_kwargs); rk["a"] = float(av)
            MR, _ = sim.compute_MR(t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
                                   kind=kind, max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose)
            rows.append({"model":"GBM","risk_type":risk_type,"t_idx":float(t_idx),
                         "param_name":"a","param_value":float(av),
                         "MR":float(MR),"MR0":float(MR0),"DeltaMR":float(MR-MR0)})
            done += 1
            if done == 1 or done % max(1, total // 10) == 0 or done == total:
                print(f"[AVERS] model=GBM | risk={risk_type} | a | t_idx={t_idx} | {100*done/total:6.2f}%")
        df = pd.DataFrame(rows)
        df.to_csv(f"sens_aversion_GBM_{risk_type}_a_t{t_idx}.csv", index=False)

# k: elw, entw
for risk_type in ["elw", "entw"]:
    base_kwargs = dict(risk_specs[risk_type])
    for t_idx in t_list:
        MR0, _ = sim.compute_MR(t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
                                kind=kind, max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose)
        rows = []
        total = len(k_grid)
        done = 0
        print(f"[AVERS] model=GBM | risk={risk_type} | param=k | t_idx={t_idx} | total={total}")
        for kv in k_grid:
            rk = dict(base_kwargs); rk["k"] = float(kv)
            MR, _ = sim.compute_MR(t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
                                   kind=kind, max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose)
            rows.append({"model":"GBM","risk_type":risk_type,"t_idx":float(t_idx),
                         "param_name":"k","param_value":float(kv),
                         "MR":float(MR),"MR0":float(MR0),"DeltaMR":float(MR-MR0)})
            done += 1
            if done == 1 or done % max(1, total // 10) == 0 or done == total:
                print(f"[AVERS] model=GBM | risk={risk_type} | k | t_idx={t_idx} | {100*done/total:6.2f}%")
        df = pd.DataFrame(rows)
        df.to_csv(f"sens_aversion_GBM_{risk_type}_k_t{t_idx}.csv", index=False)

# beta: esl
risk_type = "esl"
base_kwargs = dict(risk_specs[risk_type])
for t_idx in t_list:
    MR0, _ = sim.compute_MR(t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=base_kwargs,
                            kind=kind, max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose)
    rows = []
    total = len(beta_list)
    done = 0
    print(f"[AVERS] model=GBM | risk=esl | param=beta | t_idx={t_idx} | total={total}")
    for bv in beta_list:
        rk = dict(base_kwargs); rk["beta"] = float(bv)
        MR, _ = sim.compute_MR(t_idx=float(t_idx), risk_type=risk_type, risk_kwargs=rk,
                               kind=kind, max_iter=max_iter, tol=tol, alpha=alpha, verbose=verbose)
        rows.append({"model":"GBM","risk_type":"esl","t_idx":float(t_idx),
                     "param_name":"beta","param_value":float(bv),
                     "MR":float(MR),"MR0":float(MR0),"DeltaMR":float(MR-MR0)})
        done += 1
        print(f"[AVERS] model=GBM | esl | beta | t_idx={t_idx} | {100*done/total:6.2f}%")
    df = pd.DataFrame(rows)
    df.to_csv(f"sens_aversion_GBM_esl_beta_t{t_idx}.csv", index=False)

# -----------------------------
# Repeat for Heston and Jump by changing ONLY sim instantiation
# -----------------------------

# Heston
sim = HestonSimulator(
    M=M, N=N, S0=S0, K=S0, T=T, t0=t0,
    mu=mu, sigma=sigma,
    kappa=kappa, theta=theta, sigma_v=sigma_v, corr=corr
)
# Copy-paste the exact GBM blocks above and replace model="GBM" -> "Heston" in rows and filenames

# Jump
sim = JumpDiffusionSimulator(
    M=M, N=N, S0=S0, K=S0, T=T, t0=t0,
    mu=mu, sigma=sigma,
    lam=lam, meanJ=meanJ, stdJ=stdJ
)
# Copy-paste the exact GBM blocks above and replace model="GBM" -> "Jump" in rows and filenames


In [ ]:
# ============================================================
# BLOCO 5 - HEDGE COMPARE (t_idx=0): save after EACH (model, risk_type)
# Saves: h_opt, h_delta, MR_opt, MR_delta, rho_T_opt, rho_T_delta
# ============================================================

import numpy as np
import pandas as pd

max_iter = 20
tol = 1e-4
alpha = 1e-3
verbose = False
kind = "MinVar"

# -----------------------------
# GBM
# -----------------------------
sim = GBMSimulator(M=M, N=N, S0=S0, K=S0, T=T, t0=t0, mu=mu, sigma=sigma)

for risk_type, risk_kwargs in risk_specs.items():
    print(f"[HEDGE] model=GBM | risk={risk_type} | start")

    h_opt, _ = sim.optimize_hedge(
        risk_type=risk_type,
        risk_kwargs=risk_kwargs,
        t_idx=0.0,
        kind=kind,
        max_iter=max_iter,
        tol=tol,
        alpha=alpha,
        verbose=verbose
    )
    L_opt = sim.forward_PL(h_opt, L0=0.0, t_start=0.0)
    rho_T_opt = sim.risk_function(L_opt[:, -1], risk_type, **risk_kwargs)

    h_delta = sim.init_control(kind="Delta")
    L_delta = sim.forward_PL(h_delta, L0=0.0, t_start=0.0)
    rho_T_delta = sim.risk_function(L_delta[:, -1], risk_type, **risk_kwargs)

    df = pd.DataFrame([{
        "model": "GBM",
        "risk_type": risk_type,
        "h_opt": h_opt,
        "h_delta": h_delta,
        "MR_opt": float(rho_T_opt),
        "MR_delta": float(rho_T_delta),
        "rho_T_opt": float(rho_T_opt),
        "rho_T_delta": float(rho_T_delta),
    }])
    df.to_csv(f"hedge_compare_GBM_{risk_type}.csv", index=False)
    print(f"[HEDGE] saved hedge_compare_GBM_{risk_type}.csv | done")

# -----------------------------
# Heston
# -----------------------------
sim = HestonSimulator(
    M=M, N=N, S0=S0, K=S0, T=T, t0=t0,
    mu=mu, sigma=sigma,
    kappa=kappa, theta=theta, sigma_v=sigma_v, corr=corr
)

for risk_type, risk_kwargs in risk_specs.items():
    print(f"[HEDGE] model=Heston | risk={risk_type} | start")

    h_opt, _ = sim.optimize_hedge(
        risk_type=risk_type,
        risk_kwargs=risk_kwargs,
        t_idx=0.0,
        kind=kind,
        max_iter=max_iter,
        tol=tol,
        alpha=alpha,
        verbose=verbose
    )
    L_opt = sim.forward_PL(h_opt, L0=0.0, t_start=0.0)
    rho_T_opt = sim.risk_function(L_opt[:, -1], risk_type, **risk_kwargs)

    h_delta = sim.init_control(kind="Delta")
    L_delta = sim.forward_PL(h_delta, L0=0.0, t_start=0.0)
    rho_T_delta = sim.risk_function(L_delta[:, -1], risk_type, **risk_kwargs)

    df = pd.DataFrame([{
        "model": "Heston",
        "risk_type": risk_type,
        "h_opt": h_opt,
        "h_delta": h_delta,
        "MR_opt": float(rho_T_opt),
        "MR_delta": float(rho_T_delta),
        "rho_T_opt": float(rho_T_opt),
        "rho_T_delta": float(rho_T_delta),
    }])
    df.to_csv(f"hedge_compare_Heston_{risk_type}.csv", index=False)
    print(f"[HEDGE] saved hedge_compare_Heston_{risk_type}.csv | done")

# -----------------------------
# Jump
# -----------------------------
sim = JumpDiffusionSimulator(
    M=M, N=N, S0=S0, K=S0, T=T, t0=t0,
    mu=mu, sigma=sigma,
    lam=lam, meanJ=meanJ, stdJ=stdJ
)

for risk_type, risk_kwargs in risk_specs.items():
    print(f"[HEDGE] model=Jump | risk={risk_type} | start")

    h_opt, _ = sim.optimize_hedge(
        risk_type=risk_type,
        risk_kwargs=risk_kwargs,
        t_idx=0.0,
        kind=kind,
        max_iter=max_iter,
        tol=tol,
        alpha=alpha,
        verbose=verbose
    )
    L_opt = sim.forward_PL(h_opt, L0=0.0, t_start=0.0)
    rho_T_opt = sim.risk_function(L_opt[:, -1], risk_type, **risk_kwargs)

    h_delta = sim.init_control(kind="Delta")
    L_delta = sim.forward_PL(h_delta, L0=0.0, t_start=0.0)
    rho_T_delta = sim.risk_function(L_delta[:, -1], risk_type, **risk_kwargs)

    df = pd.DataFrame([{
        "model": "Jump",
        "risk_type": risk_type,
        "h_opt": h_opt,
        "h_delta": h_delta,
        "MR_opt": float(rho_T_opt),
        "MR_delta": float(rho_T_delta),
        "rho_T_opt": float(rho_T_opt),
        "rho_T_delta": float(rho_T_delta),
    }])
    df.to_csv(f"hedge_compare_Jump_{risk_type}.csv", index=False)
    print(f"[HEDGE] saved hedge_compare_Jump_{risk_type}.csv | done")
